Exemple de modèle et résolution du problème de Fabrication du ciment

In [4]:
#importer les packages utiles, le manager de package Pkg etant un package
import Pkg; Pkg.add("Cbc")
Pkg.add("JuMP")

   Updating registry at `~/.julia/registries/General`
######################################################################### 100,0%
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [5]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
N = 2 # nombre de ciments disponibles
c = [50,70] # prix de vente par ciment
b = [360,480] # temps de disponibilité du four et de l'atelier
A = [40 12;20 30] # temps nécessaire de calcination (four) et de broyage (atelier) par ciment

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, ciment[1:N] >= 0)

# define objective function
@objective(model, Max, sum(c[i]*ciment[i] for i in 1:N))

# define constraints
for i in 1:length(b)
    @constraint(model, sum(A[i,j]*ciment[j] for j in 1:N) <= b[i])
end

# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Nov  9 2020 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 2 (0) rows, 2 (0) columns and 4 (0) elements
0  Obj -0 Dual inf 120 (2)
0  Obj -0 Dual inf 120 (2)
2  Obj 1137.5
Optimal - objective value 1137.5
Optimal objective 1137.5 - 2 iterations time 0.002
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



In [3]:
# print solution
println("Solution obtenue:")
println("\t benefice = $(objective_value(model))")
for i in 1:N
    println("\t quantite de ciment $i = $(value(ciment[i]))")
end

Solution obtenue:
	 benefice = 1137.5
	 quantite de ciment 1 = 5.249999999999999
	 quantite de ciment 2 = 12.5


# Applications en optimisation pour l’e-commerce

## Cas particulier 1

In [6]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
NbF = 2 # nombre de fluides disponibles
NbM = 3 # nombre de magasins
NbD = 2 # nombre de demandes
c = [1 2 3; 1 3 2] # cout de chaque magasin
b = [2.5 1 2; 1 2 1] # Stocks de fluides par magasin
d = [2 0; 1 3]

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, Fluides[1:NbF, 1:NbM , 1:NbD] >= 0)

# define objective function
@objective(model, Min, sum(c .* sum(Fluides[:,:,k] for k in 1:NbD)))

# define constraints
for i in 1:NbF
    for k in 1:NbD
        @constraint(model, sum(Fluides[i,j,k] for j in 1:NbM) == d[k,i])
    end # for
end
for i in 1:NbF
    for j in 1:NbM
        @constraint(model, sum(Fluides[i,j,k] for k in 1:NbD) <= b[i,j])
    end # for
end


# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Nov  9 2020 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 6 (-4) rows, 9 (-3) columns and 15 (-9) elements
0  Obj 2.6999999 Primal inf 5.099997 (3)
4  Obj 9.5
Optimal - objective value 9.5
After Postsolve, objective 9.5, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 9.5 - 4 iterations time 0.002, Presolve 0.00
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



In [12]:
# print solution
println("la solution est : ")
println("\t coût = $(objective_value(model))")
for i in 1:NbF
    for j in 1:NbM
        for k in 1:NbD
            println("\t quantite de fluide $i du magazin $j pour la commande $k = $(value(Fluides[i,j,k]))")
        end
    end
end

la solution est : 
	 coût = 14.0
	 quantite de fluide 1 du magazin 1 pour la commande 1 = 1.0
	 quantite de fluide 1 du magazin 1 pour la commande 2 = 1.0
	 quantite de fluide 1 du magazin 2 pour la commande 1 = 1.0
	 quantite de fluide 1 du magazin 2 pour la commande 2 = 0.0
	 quantite de fluide 1 du magazin 3 pour la commande 1 = 0.0
	 quantite de fluide 1 du magazin 3 pour la commande 2 = 0.0
	 quantite de fluide 2 du magazin 1 pour la commande 1 = 0.0
	 quantite de fluide 2 du magazin 1 pour la commande 2 = 1.0
	 quantite de fluide 2 du magazin 2 pour la commande 1 = 0.0
	 quantite de fluide 2 du magazin 2 pour la commande 2 = 1.0
	 quantite de fluide 2 du magazin 3 pour la commande 1 = 0.0
	 quantite de fluide 2 du magazin 3 pour la commande 2 = 1.0


## Cas particulier 2

In [8]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
NbF = 2 # nombre de fluides disponibles
NbM = 3 # nombre de magasins
NbD = 2 # nombre de demandes
c = [1 2 3; 1 3 2] # cout de chaque magasin
b = [2.5 1 2; 1 2 1] # Stocks de fluides par magasin
d = [2 0; 1 3]

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, Fluides[1:NbF, 1:NbM , 1:NbD] >= 0, Int)

# define objective function
@objective(model, Min, sum(c .* sum(Fluides[:,:,k] for k in 1:NbD)))

# define constraints
for i in 1:NbF
    for k in 1:NbD
        @constraint(model, sum(Fluides[i,j,k] for j in 1:NbM) == d[k,i])
    end # for
end
for i in 1:NbF
    for j in 1:NbM
        @constraint(model, sum(Fluides[i,j,k] for k in 1:NbD) <= b[i,j])
    end # for
end


# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Nov  9 2020 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 9.5 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 6 rows, 9 columns (9 integer (6 of which binary)) and 17 elements
Cbc0012I Integer solution of 10 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 10, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from 10 to 10
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created

In [13]:
# print solution
println("la solution est : ")
println("\t coût = $(objective_value(model))")
for i in 1:NbF
    for j in 1:NbM
        for k in 1:NbD
            println("\t quantite de fluide $i du magazin $j pour la commande $k = $(value(Fluides[i,j,k]))")
        end
    end
end

la solution est : 
	 coût = 14.0
	 quantite de fluide 1 du magazin 1 pour la commande 1 = 1.0
	 quantite de fluide 1 du magazin 1 pour la commande 2 = 1.0
	 quantite de fluide 1 du magazin 2 pour la commande 1 = 1.0
	 quantite de fluide 1 du magazin 2 pour la commande 2 = 0.0
	 quantite de fluide 1 du magazin 3 pour la commande 1 = 0.0
	 quantite de fluide 1 du magazin 3 pour la commande 2 = 0.0
	 quantite de fluide 2 du magazin 1 pour la commande 1 = 0.0
	 quantite de fluide 2 du magazin 1 pour la commande 2 = 1.0
	 quantite de fluide 2 du magazin 2 pour la commande 1 = 0.0
	 quantite de fluide 2 du magazin 2 pour la commande 2 = 1.0
	 quantite de fluide 2 du magazin 3 pour la commande 1 = 0.0
	 quantite de fluide 2 du magazin 3 pour la commande 2 = 1.0


## Cas particulier 3

In [14]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
NbF = 2 # nombre de fluides disponibles
NbM = 3 # nombre de magasins
NbD = 2 # nombre de demandes
c = [1 2 3; 1 3 2] # cout de chaque magasin
cMag = [1 0 0 ; 0 2 1] #Couts d’expédition d’un colis entre chaque paire
b = [2.5 1 2; 1 2 1] # Stocks de fluides par magasin
d = [2 0; 1 3]



# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, Fluides[1:NbF, 1:NbM , 1:NbD] >= 0,Int)
@variable(model, Demande[1:NbM, 1:NbD],Bin)

# define objective function
@objective(model, Min, sum(c .* sum(Fluides[:,:,k] for k in 1:NbD))+ sum(cMag' .* Demande))

# define constraints
for i in 1:NbF
    for k in 1:NbD
        @constraint(model, sum(Fluides[i,j,k] for j in 1:NbM) == d[k,i])
    end # for
end

for i in 1:NbF
    for j in 1:NbM
        @constraint(model, sum(Fluides[i,j,k] for k in 1:NbD) <= b[i,j])
    end # for
end

for i in 1:NbM
    for j in 1:NbD
        @constraint(model, sum(Fluides[:,i,j])/(1+sum(d[j,:])) <= Demande[i,j])
        @constraint(model, Demande[i,j] <= sum(Fluides[:,i,j]))
    end # for
end


# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Nov  9 2020 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 10.6 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 3 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 14 rows, 13 columns (13 integer (10 of which binary)) and 38 elements
Cbc0012I Integer solution of 14 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 14, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 3 variables fixed on reduced cost
Cuts at root node changed objective from 13 to 14
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and cr

In [15]:
# print solution
println("la solution est : ")
println("\t coût = $(objective_value(model))")
for i in 1:NbF
    for j in 1:NbM
        for k in 1:NbD
            println("\t quantite de fluide $i du magazin $j pour la commande $k = $(value(Fluides[i,j,k]))")
        end
    end
end

la solution est : 
	 coût = 14.0
	 quantite de fluide 1 du magazin 1 pour la commande 1 = 1.0
	 quantite de fluide 1 du magazin 1 pour la commande 2 = 1.0
	 quantite de fluide 1 du magazin 2 pour la commande 1 = 1.0
	 quantite de fluide 1 du magazin 2 pour la commande 2 = 0.0
	 quantite de fluide 1 du magazin 3 pour la commande 1 = 0.0
	 quantite de fluide 1 du magazin 3 pour la commande 2 = 0.0
	 quantite de fluide 2 du magazin 1 pour la commande 1 = 0.0
	 quantite de fluide 2 du magazin 1 pour la commande 2 = 1.0
	 quantite de fluide 2 du magazin 2 pour la commande 1 = 0.0
	 quantite de fluide 2 du magazin 2 pour la commande 2 = 1.0
	 quantite de fluide 2 du magazin 3 pour la commande 1 = 0.0
	 quantite de fluide 2 du magazin 3 pour la commande 2 = 1.0


## Cas particulier 4

In [180]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP
using LinearAlgebra
include("Donnees_Pb2-4/2_4.jl")

# Data

# nb_prod : nombre de produits disponibles
# nb_mag : nombre de magasins
# nb_dem : nombre de demandes
# nb_noeuds : nombre de noeuds
# matrice_S : l’ensemble des smp ∈ N : stock de produit p ∈ P dans le magasin m ∈ M
# matrice_Q : l’ensemble des Qdp ∈ N : quantité de produit p ∈ P dans la commande d ∈ D.
# matrice_R : l’ensemble des rij ∈ R : valeur de l’arc allant de i vers j représentant la distance à parcourir/temps de trajet entre les sites i et j

#donnee = "Data_ppb_tournee_d5m10p10.0.txt"
#donnee = "Data_ppb_tournee_d5m10p10.1.txt"
#donnee = "Data_ppb_tournee_d5m10p10.2.txt"
#donnee = "Data_ppb_tournee_d10m10p10.0.txt"
#donnee = "Data_ppb_tournee_d10m10p10.1.txt"
#donnee = "Data_ppb_tournee_d10m10p10.2.txt"
#donnee = "Data_ppb_tournee_d20m10p10.0.txt"
#donnee = "Data_ppb_tournee_d20m10p10.1.txt"
#donnee = "Data_ppb_tournee_d20m10p10.2.txt"
#donnee = "Data_ppb_tournee_d50m10p10.0.txt"
#donnee = "Data_ppb_tournee_d50m10p10.1.txt"
#donnee = "Data_ppb_tournee_d50m10p10.2.txt"
#donnee = "Data_ppb_tournee_d100m10p10.0.txt"
#donnee = "Data_ppb_tournee_d100m10p10.1.txt"
#donnee = "Data_ppb_tournee_d100m10p10.2.txt"
#donnee = "Data_test_4_2_2.txt"
donnee = "Data_test_4_2_3.txt"
#donnee = "Data_test_5_1_2.txt"
#donnee = "Data_test_5_2_3.txt"

nb_dem, nb_prod, nb_mag, nb_noeuds, matrice_S, matrice_Q, matrice_R = read_data_24("Donnees_Pb2-4/data_2-4", donnee);

In [181]:
Matrix_R = reduce(hcat,matrice_R)
Matrix_Q = reduce(hcat,matrice_Q)
Matrix_S = reduce(hcat,matrice_S)

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, Produit[1:nb_prod, 1:nb_mag , 1:nb_dem] >= 0,Int)
@variable(model, matrice_B[1:(nb_dem+1), 1:(nb_dem+1), 1:nb_mag],Bin)


@objective(model, Min, sum(sum(matrice_B[2:nb_dem+1, 2:nb_dem+1, k] for k = 1:nb_mag).* Matrix_R[nb_mag+1:nb_noeuds, nb_mag+1:nb_noeuds])
                    + sum((matrice_B[2:nb_dem+1, 1, :] + matrice_B[1, 2:nb_dem+1, :]) .* (Matrix_R[nb_mag+1:nb_noeuds,1:nb_mag])))

# Definition des constraints
for i in 1:nb_mag
    for j in 1:nb_dem
        @constraint(model, sum(matrice_B[j+1,:,i]) <= sum(Produit[:,i,j]))
        @constraint(model, sum(Produit[:,i,j])/(1+sum(Matrix_Q[:,j]))<= sum(matrice_B[j+1,:,i]))
        @constraint(model, sum(matrice_B[:,j+1,i]) <= sum(Produit[:,i,j]))
        @constraint(model, sum(Produit[:,i,j])/(1+sum(Matrix_Q[:,j]))<= sum(matrice_B[:,j+1,i]))
    end #for
end

# define constraint 2
# Constraint 2 :
for i in 1:nb_mag
    @constraint(model, sum(matrice_B[1,2:1+nb_dem,i]) <= sum(Produit[:,i,:]))
    @constraint(model, sum(Produit[:,i,:]) /(1 + sum(Matrix_S[:,i]))<= sum(matrice_B[1,2:1+nb_dem,i]))
    @constraint(model, sum(matrice_B[2:nb_dem+1,1,i]) <= sum(Produit[:,i,:]))
    @constraint(model, sum(Produit[:,i,:]) / (1 + sum(Matrix_S[:,i]))<= sum(matrice_B[2:(nb_dem+1),1,i]))
    @constraint(model, tr(matrice_B[:,:,i]) == 0)
    
    for j in 1:1+nb_dem
        @constraint(model, sum(matrice_B[j,:,i]) <= sum(Produit[:,i,:]))
        @constraint(model, sum(Produit[:,i,:]) /(1 + sum(Matrix_S[:,i]))<= sum(matrice_B[j,:,i]))
        @constraint(model, sum(matrice_B[:,j,i]) <= sum(Produit[:,i,:]))
        @constraint(model, sum(Produit[:,i,:]) /(1 + sum(Matrix_S[:,i]))<= sum(matrice_B[:,j,i]))
    end

    for j in 1:nb_dem
        @constraint(model, sum(matrice_B[j,1:j,i])+sum(matrice_B[1:j,j,i]) <= 1)
    end
end

# define constraint 3
for i in 1:nb_prod
    for k in 1:nb_dem
        @constraint(model, sum(Produit[i,j,k] for j in 1:nb_mag) == Matrix_Q[i,k])
    end # for
end

# define constraint 5
for i in 1:nb_prod
    for j in 1:nb_mag
        @constraint(model, sum(Produit[i,j,k] for k in 1:nb_dem) <= Matrix_S[i,j])
    end # for
end

# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Nov  9 2020 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 1749.13 - 0.00 seconds
Cgl0002I 10 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0004I processed model has 83 rows, 51 columns (51 integer (42 of which binary)) and 865 elements
Cbc0031I 19 added rows had average density of 19.421053
Cbc0013I At root node, 19 cuts changed objective from 1749.1304 to 2737.8067 in 100 passes
Cbc0014I Cut generator 0 (Probing) - 4 row cuts average 17.8 elements, 0 column cuts (0 active)  in 0.014 seconds - new frequency is -100
Cbc0014I Cut generator 1 (Gomory) - 700 row cuts average 44.7 elements, 0 column cuts (0 active)  in 0.018 seconds - new frequency is 1
Cbc0014I Cut generator 2 (Knapsac

In [121]:
# print solution
println("la solution est : ")
println("\t la distance est = $(objective_value(model))")
for j in 1:nb_mag
    println("\tPour le magasin $j")
    for i in 1:nb_prod
        for k in 1:nb_dem
            println("\t\t quantite de produit $i pour la commande $k = $(value(Produit[i,j,k]))")
        end
    end
end

la solution est : 
	 la distance est = 14.0
	Pour le magasin 1
		 quantite de produit 1 pour la commande 1 = 1.0
		 quantite de produit 1 pour la commande 2 = 0.0
		 quantite de produit 1 pour la commande 3 = 2.0
		 quantite de produit 1 pour la commande 4 = 3.0
		 quantite de produit 1 pour la commande 5 = 4.0
		 quantite de produit 2 pour la commande 1 = 0.0
		 quantite de produit 2 pour la commande 2 = 3.0
		 quantite de produit 2 pour la commande 3 = 1.0
		 quantite de produit 2 pour la commande 4 = 2.0
		 quantite de produit 2 pour la commande 5 = 4.0


In [119]:
# Le trajet à suivre
for k in 1:nb_mag
    depart = 1
    println("Le trajet à suivre pour le livreur du magasin $k est :")
    print("\t M$k -> ")
    for i in 1:nb_dem+1
        for j in 1:nb_dem+1
            #println("$(value(matrice_B[depart,j,k]))")
            if value(matrice_B[depart,j,k]) == 1
                depart = j
                if depart == 1
                    println("M$k")
                else
                    print("C$(j-1) -> ")
                end
                break
            end
        end
    end
end

Le trajet à suivre pour le livreur du magasin 1 est :
	 M1 -> C1 -> C4 -> C5 -> C3 -> C2 -> M1


# Bonus

## Cas de Ciment

In [62]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data

Nb_ciment = 2 # Nombre de ciment
Nb_temps = 2 # Nombre de contraintes de temps
C = [50; 70] # Coût ciment
Temps = [40 20; 12 30]
Temps_Limite = [360; 480]
Seuil_Min = [5; 0] # Seuil Minimal Ciment
Seuil = 10



#=
# On peut utiliser des nombre aléatoire
Nb_ciment = rand((1:4))
Nb_temps = rand((1:3))
C = rand((1:100),Nb_ciment)
Temps = rand((1:50),Nb_temps,Nb_ciment)
Temps_Limite = rand((100:600),Nb_temps)
Seuil_Min = rand((1:10),Nb_ciment)
Seuil = rand((1:10))
=#

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, Ciment[1:Nb_ciment] >= 0)

# define objective function
@objective(model, Max, sum(C.*Ciment))

# define constraints Four Limité d'utilisation / jour
for i in 1:Nb_temps
    @constraint(model, sum(Temps[:,i].*Ciment) <= Temps_Limite[i])
    @constraint(model, Ciment[i] >= Seuil_Min[i])
end

# define constraints Seuil Minimal Ciment
@constraint(model, sum(Ciment) >= Seuil)


# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Nov  9 2020 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 0 (-3) rows, 0 (-4) columns and 0 (-9) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 10838
After Postsolve, objective 10838, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 10838 - 0 iterations time 0.002, Presolve 0.00
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



In [85]:
# Affichage de variable utiliser : 
println("Les variable utiliser sont :")
println("\tNb_ciment = $Nb_ciment")
println("\tNb_temps = $Nb_temps")
println("\tC = $C")
println("\tTemps = $Temps")
println("\tTemps_Limite = $Temps_Limite")
println("\tSeuil_Min = $Seuil_Min")
println("\tSeuil = $Seuil")

# print solution
println("la solution est : ")
println("\t coût = $(objective_value(model))")
for i in 1:Nb_ciment
    println("\t Quantite de Ciment de typr T$i = $(value(Ciment[i]))")
end

Les variable utiliser sont :
	Nb_ciment = 4
	Nb_temps = 1
	C = [10497, 15821]
	Temps = [3 25 19 32]
	Temps_Limite = [500]
	Seuil_Min = [3, 6, 6, 8]
	Seuil = 4
la solution est : 
	 coût = 1.4437408e7
	 Quantite de Ciment de typr T1 = 3.0
	 Quantite de Ciment de typr T2 = 163.66666666666666
	 Quantite de Ciment de typr T3 = 0.0
	 Quantite de Ciment de typr T4 = 0.0


## Cas des voitures

In [83]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data

Nb_Voitures = 2 # Nombre de Voitures
C = [10000; 9000] # Coût
Max_heure = 60
Heure = [0.06; 0.05]
Max_surface = 15000
Surface = [10; 20]
Max_Luxe = 800

#=
# On peut utiliser des nombre aléatoire
Nb_Voitures = 2
C = rand((1000:50000),Nb_Voitures)
Max_heure = rand((20:100))
Heure = rand((0.02:0.1),Nb_Voitures)
Surface = rand((5:30),Nb_Voitures)
Max_Luxe = rand((100:1000))
=#


# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, Voitures[1:Nb_Voitures] >= 0, Int)

# define objective function
@objective(model, Max, sum(C.*Voitures))

# define constraints
@constraint(model, sum(Heure.*Voitures) <= Max_heure)
@constraint(model, sum(Surface.*Voitures) <= Max_surface)
@constraint(model, Voitures[1] <= Max_Luxe)




# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Nov  9 2020 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 1.44446e+07 - 0.00 seconds
Cgl0004I processed model has 1 rows, 2 columns (2 integer (0 of which binary)) and 2 elements
Cbc0012I Integer solution of -14437408 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0006I The LP relaxation is infeasible or too expensive
Cbc0013I At root node, 0 cuts changed objective from -14444599 to -14432235 in 1 passes
Cbc0014I Cut generator 0 (Probing) - 0 row cuts average 0.0 elements, 3 column cuts (3 active)  in 0.000 seconds - new frequency is 1
Cbc0014I Cut generator 1 (Gomory) - 0 row cuts average 0.0 elements, 0 column cuts (0 active)  in 0.000 seconds - new frequency is -100
Cbc0014I Cut generator 2 (Knapsack) - 0 row cuts average 0.0 elements, 0 column cuts (0 active)  in 0.000 seconds - new frequency is -100
Cbc0014I Cut generator 3 (Clique) - 0 row cut

In [84]:
# Affichage de variable utiliser : 
println("Les variable utiliser sont :")
println("\tNb_Voitures = $Nb_Voitures")
println("\tC = $C")
println("\tMax_heure = $Max_heure")
println("\tHeure = $Heure")
println("\tSurface = $Surface")
println("\tMax_Luxe = $Max_Luxe")

# print solution
println("la solution est : ")
println("\t coût = $(objective_value(model))")
println("\t Quantite de Voiture Luxe = $(value(Voitures[1]))")
println("\t Quantite de Voiture Standard = $(value(Voitures[2]))")


Les variable utiliser sont :
	Nb_Voitures = 2
	C = [10497, 15821]
	Max_heure = 76
	Heure = [0.02, 0.02]
	Surface = [8, 22]
	Max_Luxe = 771
la solution est : 
	 coût = 1.4437408e7
	 Quantite de Voiture Luxe = 771.0
	 Quantite de Voiture Standard = 401.0


## Cas des teintures

In [110]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
Nb_Teinture = 2 # Nombre de teinture
Nb_seuil = 3
C = [10; 20] # Coût teinture
cout_Seuil_Min = [500 150 20 ;400 50 0]
cout_Seuil_Max = [20;0]
Seuil_Min = [500; 100; 5] # Seuil Minimal
Seuil_Max = 15


#=
# On peut utiliser des nombre aléatoire
Nb_Teinture = rand((2:5))
Nb_seuil = rand((1:5))
C = rand((0:50),Nb_Teinture)
cout_Seuil_Min = rand((0:1000),Nb_seuil,Nb_Teinture)
cout_Seuil_Max = rand((0:50),Nb_Teinture)
Seuil_Min = rand((1:1000),Nb_seuil)
Seuil_Max = rand((10:20))
=#


# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, Teinture[1:Nb_Teinture] >= 0)

# define objective function
@objective(model, Min, sum(C.*Teinture))

# define constraints  Seuil Minimal
for i in 1:Nb_seuil
    @constraint(model, sum(cout_Seuil_Min[:,i].*Teinture) >= Seuil_Min[i])
end

# define constraints Seuil Maximal
@constraint(model, sum(cout_Seuil_Max.*Teinture) <= Seuil_Max)

# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Nov  9 2020 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 2 (-2) rows, 2 (0) columns and 4 (-2) elements
0  Obj 6.7499998 Primal inf 0.92583134 (2)
1  Obj 13.75
Optimal - objective value 13.75
After Postsolve, objective 13.75, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 13.75 - 1 iterations time 0.002, Presolve 0.00
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



In [111]:
# Affichage de variable utiliser : 
println("Les variable utiliser sont :")
println("\tNb_Teinture = $Nb_Teinture")
println("\tNb_seuil = $Nb_seuil")
println("\tC = $C")
println("\tcout_Seuil_Min = $cout_Seuil_Min")
println("\tcout_Seuil_Max = $cout_Seuil_Max")
println("\tSeuil_Min = $Seuil_Min")
println("\tSeuil_Max = $Seuil_Max")

# print solution
println("la solution est : ")
println("\t coût = $(objective_value(model))")
for i in 1:Nb_Teinture
    println("\t Quantite du produit I$i = $(value(Teinture[i]))")
end

Les variable utiliser sont :
	Nb_Teinture = 2
	Nb_seuil = 3
	C = [10, 20]
	cout_Seuil_Min = [500 150 20; 400 50 0]
	cout_Seuil_Max = [20, 0]
	Seuil_Min = [500, 100, 5]
	Seuil_Max = 15
la solution est : 
	 coût = 13.75
	 Quantite du produit I1 = 0.75
	 Quantite du produit I2 = 0.3125


## Cas des Armoiries et tables

In [103]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data

Nb_Var = 2 # Nombre de Variables (Table, Armoires)
C = [8; 5] # Coût
cout_Materiaux = [9;5]
cout_Temps = [1;1]
Materiaux = 45
Temps_dispo = 6


# On peut utiliser des nombre aléatoire
#=
Nb_Var = 2
C = rand((0:20),Nb_Var)
cout_Materiaux = rand((0:20),Nb_Var)
cout_Temps = rand((0:50),Nb_Var)
Materiaux = rand((30:100))
Temps_dispo = rand((4:20))
=#

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, Produit[1:Nb_Var] >= 0, Int)

# define objective function
@objective(model, Max, sum(C.*Produit))

# define constraints
@constraint(model, sum(cout_Materiaux.*Produit) <= Materiaux)
@constraint(model, sum(cout_Temps.*Produit) <= Temps_dispo)


# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Nov  9 2020 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 2.22222 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 0 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
MixedIntegerRounding2 was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
FlowCover was tried 0 times and cr

In [104]:
# Affichage de variable utiliser : 
println("Les variable utiliser sont :")
println("\tC = $C")
println("\tcout_Materiaux = $cout_Materiaux")
println("\tcout_Temps = $cout_Temps")
println("\tMateriaux = $Materiaux")
println("\tTemps_dispo = $Temps_dispo")

# print solution
println("la solution est : ")
println("\t coût = $(objective_value(model))")
println("\t Quantite d'armoire = $(value(Produit[1]))")
println("\t Quantite de table = $(value(Produit[2]))")

Les variable utiliser sont :
	C = [12, 15]
	cout_Materiaux = [6, 15]
	cout_Temps = [42, 27]
	Materiaux = 91
	Temps_dispo = 4
la solution est : 
	 coût = 0.0
	 Quantite d'armoire = 0.0
	 Quantite de table = 0.0


## Cas de la bourse

In [176]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data

Nb_Var = 6 # Nombre de Variables
C = [0.07; 0.1; 0.19; 0.12; 0.08; 0.14] # Coût
Seuil_Max = [0.25;0.25;0.25;0.25;0.25;0.25]
Cout_Risque = [1.7; 1.2; 3.7; 2.4; 2.0; 2.9]
Risque = 2
Limitation = 5000000
Metaux_speciaux = 0.3
Investissement_fiable = 0.45


# On peut utiliser des nombre aléatoire
#=
Nb_Var = 6
C = rand((0:100)/100,Nb_Var)
Seuil_Max = rand((0:100)/100,Nb_Var)
Cout_Risque = rand((10:50)/10,Nb_Var)
Risque = rand((10000:500000))
Limitation = rand(Int)
Metaux_speciaux = rand((0:100)/100)
Investissement_fiable = rand(0:100)/100
=#


# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, Variables[1:Nb_Var] >= 0)

# define objective function
@objective(model, Max, sum(C.*Variables))


# define constraints

@constraint(model, sum(Cout_Risque.*Variables) <= Risque*sum(Variables))

@constraint(model, Variables[3]+Variables[4] >= Metaux_speciaux*sum(Variables))

@constraint(model, Variables[1]+Variables[2] >= Investissement_fiable*sum(Variables))

for i in 1:Nb_Var
    @constraint(model, Variables[i] <= Seuil_Max[i]*sum(Variables))
end # for
@constraint(model, sum(Variables) <= Limitation)


# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Nov  9 2020 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 10 (0) rows, 6 (0) columns and 59 (0) elements
0  Obj -0 Dual inf 0.6999994 (6)
0  Obj -0 Dual inf 0.6999994 (6)
11  Obj 520000
Optimal - objective value 520000
Optimal objective 520000 - 11 iterations time 0.002
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



In [177]:
# Affichage de variable utiliser : 
println("Les variable utiliser sont :")
println("\tC = $C")
println("\tSeuil_Max = $Seuil_Max")
println("\tCout_Risque = $Cout_Risque")
println("\tRisque = $Risque")
println("\tLimitation = $Limitation")
println("\tMetaux_speciaux = $Metaux_speciaux")
println("\tInvestissement_fiable = $Investissement_fiable")

# print solution
println("la solution est : ")
println("\t coût = $(objective_value(model))")
for i in 1:Nb_Var
    println("\t L'investissement au produit financier $i = $(value(Variables[i]))")
end

Les variable utiliser sont :
	C = [0.07, 0.1, 0.19, 0.12, 0.08, 0.14]
	Seuil_Max = [0.25, 0.25, 0.25, 0.25, 0.25, 0.25]
	Cout_Risque = [1.7, 1.2, 3.7, 2.4, 2.0, 2.9]
	Risque = 2
	Limitation = 5000000
	Metaux_speciaux = 0.3
	Investissement_fiable = 0.45
la solution est : 
	 coût = 520000.0
	 L'investissement au produit financier 1 = 1.25e6
	 L'investissement au produit financier 2 = 1.25e6
	 L'investissement au produit financier 3 = 250000.0
	 L'investissement au produit financier 4 = 1.25e6
	 L'investissement au produit financier 5 = 500000.00000000006
	 L'investissement au produit financier 6 = 500000.0000000002
